### 3. Preprocessing

In [4]:
from PIL import Image
import os
import shutil
import random
from pathlib import Path

In [3]:
input_dir = Path("data")
output_dir = Path("data_preprocessed")
target_size = (224, 224)

for class_name in os.listdir(input_dir):
    class_input_path = input_dir / class_name
    class_output_path = output_dir / class_name
    class_output_path.mkdir(parents=True, exist_ok=True)

    for file_name in os.listdir(class_input_path):
        input_file = class_input_path / file_name
        output_file = class_output_path / file_name

        try:
            with Image.open(input_file) as img:
                img = img.convert("RGB")  # falls PNG mit Transparenz
                img = img.resize(target_size, Image.Resampling.LANCZOS)
                img.save(output_file)
        except Exception as e:
            print(f"Fehler bei {input_file}: {e}")

⚠️ Fehler bei data/Burrito/.ipynb_checkpoints: [Errno 21] Is a directory: 'data/Burrito/.ipynb_checkpoints'
⚠️ Fehler bei data/Muffin/.ipynb_checkpoints: [Errno 21] Is a directory: 'data/Muffin/.ipynb_checkpoints'
⚠️ Fehler bei data/Hot dog/.ipynb_checkpoints: [Errno 21] Is a directory: 'data/Hot dog/.ipynb_checkpoints'


In [5]:
def balance_by_oversampling(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    class_paths = {cls: list(Path(input_dir, cls).glob("*.jpg")) for cls in os.listdir(input_dir)}
    
    # Zielgröße: so viele Bilder wie die größte Klasse
    max_len = max(len(paths) for paths in class_paths.values())
    
    for cls, paths in class_paths.items():
        target_dir = Path(output_dir, cls)
        target_dir.mkdir(parents=True, exist_ok=True)
        
        copies_needed = max_len - len(paths)
        all_images = paths.copy()
        
        # Einfach random duplizieren
        for i in range(copies_needed):
            src = random.choice(paths)
            dst = target_dir / f"aug_{i}_{src.name}"
            shutil.copy(src, dst)
        
        # Originalbilder auch reinlegen
        for src in all_images:
            shutil.copy(src, target_dir / src.name)

In [7]:
balance_by_oversampling("data_split/train", "data_split/train_balanced")